In [1]:
import math
import numpy as np

In [2]:
def to_tableau(c, A, b):
    xb = [eq + [x] for eq, x in zip(A, b)]
    z = c + [0]
    return xb + [z]

In [3]:
def can_be_improved(tableau):
    z = tableau[-1]
    return any(x > 0 for x in z[:-1])

In [4]:
def get_pivot_position(tableau):
    z = tableau[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)
    
    restrictions = []
    for eq in tableau[:-1]:
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)
        
    if (all([r == math.inf for r in restrictions])):
        raise Exception("Linear program is unbounded.")

    row = restrictions.index(min(restrictions))
    print(row, column)
    return row, column

In [5]:
def pivot_step(tableau, pivot_position):
    new_tableau = [[] for eq in tableau]
    
    i, j = pivot_position
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value
    
    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier
    print(new_tableau)
    return new_tableau

In [6]:
def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

In [7]:
def get_solution(tableau):
    columns = np.array(tableau).T
    solutions = []
    print(columns.T)
    for column in columns[:-1]:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)
    
    return solutions

In [8]:
def simplex(c, A, b):
    tableau = to_tableau(c, A, b)

    while can_be_improved(tableau):
        pivot_position = get_pivot_position(tableau)
        tableau = pivot_step(tableau, pivot_position)

    return get_solution(tableau)

In [9]:
def get_objective_function_value(tableau):
    return -tableau[-1][-1]

In [10]:
def to_objective_function_value(c, solution):
    return sum(np.array(c) * np.array(solution))

In [11]:
def can_be_improved_for_dual(tableau):
    rhs_entries = [row[-1] for row in tableau[:-1]]
    return any([entry < 0 for entry in rhs_entries])

In [12]:
def get_pivot_position_for_dual(tableau):
    rhs_entries = [row[-1] for row in tableau[:-1]]
    min_rhs_value = min(rhs_entries)
    row = rhs_entries.index(min_rhs_value)
    
    columns = []
    for index, element in enumerate(tableau[row][:-1]):
        if element < 0:
            columns.append(index)
    columns_values = [tableau[row][c] / tableau[-1][c] for c in columns]
    column_min_index = columns_values.index(min(columns_values))
    column = columns[column_min_index]

    return row, column

In [13]:
def dual_simplex(c, A, b):
    tableau = to_tableau(c, A, b)

    while can_be_improved_for_dual(tableau):
        pivot_position = get_pivot_position_for_dual(tableau)
        tableau = pivot_step(tableau, pivot_position)

    return get_solution(tableau)

In [14]:
A = [[1, 2, -2, 0],
     [0, 1, 3, 1]]
b = [2, 5]
c = [0, 3, 1, 0]

In [15]:
solution = simplex(c, A, b)
print('solution: ', solution)

0 1
[array([ 0.5,  1. , -1. ,  0. ,  1. ]), array([-0.5,  0. ,  4. ,  1. ,  4. ]), array([-1.5,  0. ,  4. ,  0. , -3. ])]
1 2
[array([0.375, 1.   , 0.   , 0.25 , 2.   ]), array([-0.125,  0.   ,  1.   ,  0.25 ,  1.   ]), array([-1.,  0.,  0., -1., -7.])]
[[ 0.375  1.     0.     0.25   2.   ]
 [-0.125  0.     1.     0.25   1.   ]
 [-1.     0.     0.    -1.    -7.   ]]
solution:  [0, 2.0, 1.0, 0]


In [16]:
primal = to_objective_function_value(c, simplex(c, A, b))
print('Primal: ', primal)

0 1
[array([ 0.5,  1. , -1. ,  0. ,  1. ]), array([-0.5,  0. ,  4. ,  1. ,  4. ]), array([-1.5,  0. ,  4. ,  0. , -3. ])]
1 2
[array([0.375, 1.   , 0.   , 0.25 , 2.   ]), array([-0.125,  0.   ,  1.   ,  0.25 ,  1.   ]), array([-1.,  0.,  0., -1., -7.])]
[[ 0.375  1.     0.     0.25   2.   ]
 [-0.125  0.     1.     0.25   1.   ]
 [-1.     0.     0.    -1.    -7.   ]]
Primal:  7.0


In [17]:
dual = to_objective_function_value(c, dual_simplex(c, A, b))
print('Dual: ', dual)

[[ 1  2 -2  0  2]
 [ 0  1  3  1  5]
 [ 0  3  1  0  0]]
Dual:  0


In [18]:
A = [[1, 4, 0, 0, -1, 1],
     [0, -3, 1, 0, 5, -1],
     [0, -2, 0, 1, 2, 0.5]]
b = [0.5, 1.25, 0.625]
c = [0, -1, 0, 0, -3, 1]

In [19]:
solution = simplex(c, A, b)
print('solution: ', solution)

0 5
[array([ 1. ,  4. ,  0. ,  0. , -1. ,  1. ,  0.5]), array([1.  , 1.  , 1.  , 0.  , 4.  , 0.  , 1.75]), array([-0.5  , -4.   ,  0.   ,  1.   ,  2.5  ,  0.   ,  0.375]), array([-1. , -5. ,  0. ,  0. , -2. ,  0. , -0.5])]
[[ 1.     4.     0.     0.    -1.     1.     0.5  ]
 [ 1.     1.     1.     0.     4.     0.     1.75 ]
 [-0.5   -4.     0.     1.     2.5    0.     0.375]
 [-1.    -5.     0.     0.    -2.     0.    -0.5  ]]
solution:  [0, 0, 1.75, 0.375, 0, 0.5]


In [20]:
primal = to_objective_function_value(c, simplex(c, A, b))
print('Primal: ', primal)

0 5
[array([ 1. ,  4. ,  0. ,  0. , -1. ,  1. ,  0.5]), array([1.  , 1.  , 1.  , 0.  , 4.  , 0.  , 1.75]), array([-0.5  , -4.   ,  0.   ,  1.   ,  2.5  ,  0.   ,  0.375]), array([-1. , -5. ,  0. ,  0. , -2. ,  0. , -0.5])]
[[ 1.     4.     0.     0.    -1.     1.     0.5  ]
 [ 1.     1.     1.     0.     4.     0.     1.75 ]
 [-0.5   -4.     0.     1.     2.5    0.     0.375]
 [-1.    -5.     0.     0.    -2.     0.    -0.5  ]]
Primal:  0.5


In [21]:
dual = to_objective_function_value(c, dual_simplex(c, A, b))
print('Dual: ', dual)

[[ 1.     4.     0.     0.    -1.     1.     0.5  ]
 [ 0.    -3.     1.     0.     5.    -1.     1.25 ]
 [ 0.    -2.     0.     1.     2.     0.5    0.625]
 [ 0.    -1.     0.     0.    -3.     1.     0.   ]]
Dual:  0.0


In [22]:
A = [[-2, 1, 1, 1, 0, 0],
     [1, -1, 0, 0, 1, 0],
     [2, -3, -1, 0, 0, 1]]
b = [1, 2, 6]
c = [2, 1, -1, 0, 0, 0]

In [23]:
try: 
    simplex(c, A, b)
except Exception as e:
    print(e)

1 0
[array([ 0., -1.,  1.,  1.,  2.,  0.,  5.]), array([ 1., -1.,  0.,  0.,  1.,  0.,  2.]), array([ 0., -1., -1.,  0., -2.,  1.,  2.]), array([ 0.,  3., -1.,  0., -2.,  0., -4.])]
Linear program is unbounded.


In [24]:
dual = to_objective_function_value(c, dual_simplex(c, A, b))
print('Dual: ', dual)

[[-2  1  1  1  0  0  1]
 [ 1 -1  0  0  1  0  2]
 [ 2 -3 -1  0  0  1  6]
 [ 2  1 -1  0  0  0  0]]
Dual:  0


In [25]:
A = [[1, 2, 2, 1, 1, 0],
     [0, 1, 1, 2, 0, 1]]
b = [6, 10]
c = [1, 2, 3, 2, 0, 0]

In [26]:
solution = simplex(c, A, b)
print('solution: ', solution)

0 0
[array([1., 2., 2., 1., 1., 0., 6.]), array([ 0.,  1.,  1.,  2.,  0.,  1., 10.]), array([ 0.,  0.,  1.,  1., -1.,  0., -6.])]
0 2
[array([0.5, 1. , 1. , 0.5, 0.5, 0. , 3. ]), array([-0.5,  0. ,  0. ,  1.5, -0.5,  1. ,  7. ]), array([-0.5, -1. ,  0. ,  0.5, -1.5,  0. , -9. ])]
1 3
[array([ 0.66666667,  1.        ,  1.        ,  0.        ,  0.66666667,
       -0.33333333,  0.66666667]), array([-0.33333333,  0.        ,  0.        ,  1.        , -0.33333333,
        0.66666667,  4.66666667]), array([ -0.33333333,  -1.        ,   0.        ,   0.        ,
        -1.33333333,  -0.33333333, -11.33333333])]
[[  0.66666667   1.           1.           0.           0.66666667
   -0.33333333   0.66666667]
 [ -0.33333333   0.           0.           1.          -0.33333333
    0.66666667   4.66666667]
 [ -0.33333333  -1.           0.           0.          -1.33333333
   -0.33333333 -11.33333333]]
solution:  [0, 0, 0.6666666666666665, 4.666666666666667, 0, 0]


In [27]:
primal = to_objective_function_value(c, simplex(c, A, b))
print('Primal: ', primal)

0 0
[array([1., 2., 2., 1., 1., 0., 6.]), array([ 0.,  1.,  1.,  2.,  0.,  1., 10.]), array([ 0.,  0.,  1.,  1., -1.,  0., -6.])]
0 2
[array([0.5, 1. , 1. , 0.5, 0.5, 0. , 3. ]), array([-0.5,  0. ,  0. ,  1.5, -0.5,  1. ,  7. ]), array([-0.5, -1. ,  0. ,  0.5, -1.5,  0. , -9. ])]
1 3
[array([ 0.66666667,  1.        ,  1.        ,  0.        ,  0.66666667,
       -0.33333333,  0.66666667]), array([-0.33333333,  0.        ,  0.        ,  1.        , -0.33333333,
        0.66666667,  4.66666667]), array([ -0.33333333,  -1.        ,   0.        ,   0.        ,
        -1.33333333,  -0.33333333, -11.33333333])]
[[  0.66666667   1.           1.           0.           0.66666667
   -0.33333333   0.66666667]
 [ -0.33333333   0.           0.           1.          -0.33333333
    0.66666667   4.66666667]
 [ -0.33333333  -1.           0.           0.          -1.33333333
   -0.33333333 -11.33333333]]
Primal:  11.333333333333334


In [28]:
dual = to_objective_function_value(c, dual_simplex(c, A, b))
print('Dual: ', dual)

[[ 1  2  2  1  1  0  6]
 [ 0  1  1  2  0  1 10]
 [ 1  2  3  2  0  0  0]]
Dual:  0
